In [130]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

####################### Initialization #########################
# first stage parameters 
i = np.array([7, 12, 25])  # cost of installing CPU, GPU, TPU
s = np.array([1, 5, 4]) # occupied space by the servers
B = 1100
S = 90

# second stage parameters
k = 100 # number of scenarios
# create k scenarios 
d_xi = np.array([np.random.normal(75,8,k), np.random.normal(8,3,k), np.random.normal(18,2,k)])
c = np.array([8, 20, 23]) # installation cost
p = np.array([1.0/k]*k) # probability of each scenario

####################### Formulation #########################
model = gp.Model('2SP')

# decison variables: x1,x2,x3 and y1i, y2i, y3i forall i=1,...,k
x = model.addMVar((3,),  lb=0, vtype=GRB.INTEGER, name=['p-server 1','p-server 2','p-server 3'])
y = model.addMVar((3,k), lb=0, vtype=GRB.INTEGER, name='v-server')

# objective function
model.setObjective(i@x + gp.quicksum(c @ y[:,k]*p[k] for k in range(k)), GRB.MINIMIZE)

# constraint
model.addConstr(i@x <= B, name='budgetConst') # budget only for first decision
model.addConstr(s@x <= S, name='spaceConst') # spcae only for first decision
model.addConstrs((x + y[:,k] >= d_xi[:,k] for k in range(k)), 'demandConst')

####################### optimization #########################
model.optimize()

# Show results
if model.status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('\nFaced infeasible/undounded solution space.')
else:
    print('\nGot the best optimal solution. Minimum cost is: {}'.format(model.ObjVal))
    print('The first decisions are:')
    for v in model.getVars()[0:3]:
        print('{} = {} units'.format(v.VarName, v.X))


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD A8-7100 Radeon R5, 8 Compute Cores 4C+4G, instruction set [SSE2|AVX]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 302 rows, 303 columns and 606 nonzeros
Model fingerprint: 0x8daa6a43
Variable types: 0 continuous, 303 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [8e-02, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 1e+03]
Found heuristic solution: objective 1216.2600000
Presolve removed 250 rows and 249 columns
Presolve time: 0.00s
Presolved: 52 rows, 54 columns, 105 nonzeros
Variable types: 0 continuous, 54 integer (0 binary)

Root relaxation: objective 1.113100e+03, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0